In [1]:
import pandas as pd
import numpy as np

from numpy.linalg import lstsq
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

from numba import njit

In [2]:
G = 3

In [3]:
from linearmodels.datasets import wage_panel
wage_panel_df = wage_panel.load()

wage_panel_df.set_index(['nr', 'year'], inplace=True)
y = wage_panel_df['lwage']
x = wage_panel_df.drop(columns=["occupation", "lwage", "black", "hisp", "educ"])
display(x)
display(y)

exper  hours  married  union  expersq
nr    year                                       
13    1980      1   2672        0      0        1
      1981      2   2320        0      1        4
      1982      3   2940        0      0        9
      1983      4   2960        0      0       16
      1984      5   3071        0      0       25
...           ...    ...      ...    ...      ...
12548 1983      8   2080        1      0       64
      1984      9   2080        1      1       81
      1985     10   2080        1      0      100
      1986     11   2080        1      1      121
      1987     12   3380        1      1      144

[4360 rows x 5 columns]

nr     year
13     1980    1.197540
       1981    1.853060
       1982    1.344462
       1983    1.433213
       1984    1.568125
                 ...   
12548  1983    1.591879
       1984    1.212543
       1985    1.765962
       1986    1.745894
       1987    1.466543
Name: lwage, Length: 4360, dtype: float64

In [4]:
N = x.index.get_level_values(0).nunique()
T = x.index.get_level_values(1).nunique()
K = x.shape[1]
N, T, K

(545, 8, 5)

In [5]:
x = x.values.reshape(N, T, K)
y = y.values.reshape(N, T, 1)
x, y

(array([[[   1, 2672,    0,    0,    1],
         [   2, 2320,    0,    1,    4],
         [   3, 2940,    0,    0,    9],
         ...,
         [   6, 2864,    0,    0,   36],
         [   7, 2994,    0,    0,   49],
         [   8, 2640,    0,    0,   64]],
 
        [[   4, 2484,    0,    0,   16],
         [   5, 2804,    0,    0,   25],
         [   6, 2530,    0,    0,   36],
         ...,
         [   9, 2164,    0,    0,   81],
         [  10, 2749,    0,    0,  100],
         [  11, 2476,    0,    0,  121]],
 
        [[   4, 2332,    1,    0,   16],
         [   5, 2116,    1,    0,   25],
         [   6, 2500,    1,    0,   36],
         ...,
         [   9, 2340,    1,    0,   81],
         [  10, 2340,    1,    0,  100],
         [  11, 2340,    1,    0,  121]],
 
        ...,
 
        [[   4, 2008,    1,    0,   16],
         [   5, 3190,    0,    0,   25],
         [   6, 2584,    0,    0,   36],
         ...,
         [   9, 2290,    1,    0,   81],
         [  10, 31

In [7]:
# # Compute theta
# # theta, ssr, rank, singular = lstsq(x_stack, , rcond=None)
# iterations = 10000
# objective_value = np.inf
# tolerance = 1e-8
# G = 3

# x_stack = x.reshape(-1, K)
# y_stack = y.reshape(-1, 1)

# num_start_vars = x.shape[1] + G
# starting_vals = np.random.choice(N, num_start_vars, replace=False)
# x_stack_start = x[starting_vals].reshape(-1, K)
# y_stack_start = y[starting_vals].reshape(-1, 1)

# # NOTE in the paper this should be done on just a subset of the data for initialization
# theta, ssr, rank, singular = lstsq(x_stack_start, y_stack_start, rcond=None)
# residuals = y_stack - x_stack @ theta
# # residuals.reshape(N, T)
# theta

# random_draws = np.random.choice(N, size=G, replace=False)
# alpha = np.squeeze(y[random_draws] - x[random_draws, :, :] @ theta)
# alpha

# res = np.squeeze(y - x @ theta)
# euclidean_distance_between_grouping = ((res[None, :, :] - alpha[:, None, :]) ** 2).sum(axis=2)
# # FIXME some code should check if all values are included, and should create a new grouping if not
# g = np.argmin(euclidean_distance_between_grouping, axis=0)  # Closest group
# g

# res = np.squeeze(y - x @ theta)

# counts = np.bincount(g)[:, None]  # (G, 1) — number of elements in each group
# sums = np.zeros((G, res.shape[1]))  # (G, K) — sum of residuals per group
# np.add.at(sums, g, res)  # sums[i] += res[j] for all j where g[j] == i
# alpha = sums / counts  # mean = sum / count
# alpha

# obj_val_array = np.zeros(iterations)

# for i in range(iterations):
#     # Compute alpha
#     counts = np.bincount(g)[:, None]  # (G, 1) — number of elements in each group
#     sums = np.zeros((G, res.shape[1]))  # (G, K) — sum of residuals per group
#     np.add.at(sums, g, res)  # sums[i] += res[j] for all j where g[j] == i
#     alpha = sums / counts  # mean = sum / count

#     # Compute theta
#     theta, ssr, rank, singular = lstsq(x_stack, y_stack - alpha[g].reshape(-1, 1), rcond=None) # TODO check if this makes sense

#     # Compute groupings
#     res = np.squeeze(y - x @ theta)
#     euclidean_distance_between_grouping = ((res[None, :, :] - alpha[:, None, :]) ** 2).sum(axis=2)
#     # FIXME some code should check if all values are included, and should create a new grouping if not
#     g = np.argmin(euclidean_distance_between_grouping, axis=0)  # Closest group

#     # Check for convergence
#     new_objective_value = ((res - alpha[g])**2).sum()
#     if abs(objective_value - new_objective_value) < tolerance:
#         break
#     objective_value = new_objective_value

#     obj_val_array[i] = objective_value

# display(i)
# display(objective_value)
# display(g)
# display(alpha)
# display(theta)
# # Recompute alpha

In [7]:
# @njit
def _get_starting_values(y, x, G: int, N: int, K: int):
    """Generates the starting values of theta"""
    num_start_vars: int = x.shape[1] + G # FIXME I believe that shape is slow in Cython
    random_draws_theta = np.random.choice(N, num_start_vars, replace=False)
    x_stack_start = x[random_draws_theta].reshape(-1, K)
    y_stack_start = y[random_draws_theta].reshape(-1, 1)

    # FIXME some errors may arise, maybe add some checks
    theta_init = lstsq(x_stack_start, y_stack_start, rcond=None)[0]

    random_draws_alpha = np.random.choice(N, size=G, replace=False)
    alpha_init = np.squeeze(y[random_draws_alpha] - x[random_draws_alpha, :, :] @ theta_init)

    return theta_init, alpha_init

@njit
def _compute_groupings(res, alpha):
    """Computes the groupings based on the residuals and alpha"""
    euclidean_distance_between_grouping = ((res[None, :, :] - alpha[:, None, :]) ** 2).sum(axis=2)
    g = np.argmin(euclidean_distance_between_grouping, axis=0)  # Closest group
    return g

# @njit
def _compute_alpha(res, g, G):
    """Computes the alpha values based on the residuals and groupings"""
    counts = np.bincount(g, minlength=G)[:, None]  # (G, 1) — number of elements in each group
    sums = np.zeros((G, res.shape[1]))  # (G, K) — sum of residuals per group
    np.add.at(sums, g, res)  # sums[i] += res[j] for all j where g[j] == i
    alpha = sums / counts  # mean = sum / count
    return alpha

# @njit
def _compute_theta(x, y, alpha, g):
    """Computes the theta values based on the x, y, alpha and groupings"""
    # FIXME check if this makes sense
    K = x.shape[2] # FIXME I believe that shape is slow in Cython
    theta = lstsq(x.reshape(-1, K), y.reshape(-1, 1) - alpha[g].reshape(-1, 1), rcond=None)[0]
    return theta

# @njit
def _compute_residuals(y, x, theta):
    """Computes the residuals based on y, x and theta"""
    res = np.squeeze(y - x @ theta)
    return res

# @njit
def _compute_objective_value(res, alpha, g):
    """Computes the objective value based on the residuals, alpha and groupings"""
    # print(f"res: {res}")
    # print(f"alpha: {alpha}")
    # display(f"g: {g}")
    # display(f"alpha[g]: {alpha[g]}")
    objective_value = ((res - alpha[g])**2).sum()
    return objective_value

def _reorder_groups(g, alpha):
    """Reorders the groups based on the first value of alpha"""
    # FIXME this is not the best way to do this
    # But it works for now
    mapping = np.argsort(alpha[:, 0])
    ordered_g = np.argsort(mapping)[g]
    ordered_alpha = alpha[mapping]
    return ordered_g, ordered_alpha

def _grouped_fixed_effects_iteration(y, x, G: int, N: int, K: int, max_iter=10000, tol=1e-8):
    # FIXME possibly create some sort of array that stores these values
    # Could be used for debugging
    theta, alpha = _get_starting_values(y, x, G, N, K)
    res = _compute_residuals(y, x, theta)
    g = _compute_groupings(res, alpha)

    objective_value = np.inf

    iterations_used = 0
    for i in range(max_iter):
        alpha = _compute_alpha(res, g, G)
        theta = _compute_theta(x, y, alpha, g)
        res = _compute_residuals(y, x, theta)
        alpha = _compute_alpha(res, g, G)
        g = _compute_groupings(res, alpha)
        new_objective_value = _compute_objective_value(res, alpha, g)


        if abs(objective_value - new_objective_value) < tol:
            iterations_used = i
            objective_value = new_objective_value
            break

        objective_value = new_objective_value

    return theta, alpha, g, iterations_used, objective_value

def _compute_eta(y_bar, x_bar, theta):
    """Computes the eta values based on y_bar, x_bar and theta"""
    eta = np.squeeze(y_bar - x_bar @ theta)
    return eta

# FIXME not used right now but still neccesary
def _compute_statistics(objective_value, N, T, K, G):
    """Computes the statistics based on the objective value, N, T, K and G"""
    # FIXME this is not the best way to do this
    # But it works for now
    sigma_squared = 1 / (N * T - G * T - N - K) * objective_value
    BIC = 1/(N*T) * objective_value + sigma_squared * (G * T + N + K) / (N * T)
    return sigma_squared, BIC


def grouped_fixed_effects(y, x, G, max_iter=10000, tol=1e-8, gfe_iterations=20, unit_specific_effects=False):
    """
    Computes the grouped fixed effects using the algorithm described in the paper.
    """
    # FIXME not really required if unit_specific_effects is False
    # But this seems like the easiest implementation
    x_bar = np.mean(x, axis=1, keepdims=True)
    y_bar = np.mean(y, axis=1, keepdims=True)

    best_theta = None
    best_alpha = None
    best_g = None
    best_iterations_used = None
    best_objective_value = np.inf

    N = x.shape[0]
    K = x.shape[2]

    # FIXME this code does not drop constant binary variables by itself
    # The input class should be able to do this
    if unit_specific_effects:
        # Demean x and y
        x = x - x_bar
        y = y - y_bar

    for _ in range(gfe_iterations):
        theta, alpha, g, iterations_used, objective_value = _grouped_fixed_effects_iteration(
            y,
            x,
            G,
            N,
            K,
            max_iter,
            tol,
        )

        if objective_value < best_objective_value:
            best_objective_value = objective_value
            best_theta = theta
            best_g, best_alpha = _reorder_groups(g, alpha)
            # best_g, best_alpha = g, alpha
            best_iterations_used = iterations_used

    if unit_specific_effects:
        eta = _compute_eta(y_bar, x_bar, best_theta)
        return best_theta, best_alpha, best_g, eta, best_iterations_used, best_objective_value

    # NOTE None is for lack of unit specific effects
    return best_theta, best_alpha, best_g, None, best_iterations_used, best_objective_value


# FIXME Somewhere I have assumed that the sum of alpha is 0, this however does not have to be the case
# I however do not know where and it may be for the computation of eta, in which case it is not a problem.
grouped_fixed_effects(y[:30], x[:30], 3, max_iter=10000, tol=1e-8, gfe_iterations=100, unit_specific_effects=False)

(array([[ 2.09575509e-01],
        [ 5.12225237e-06],
        [-6.68369353e-02],
        [ 5.89767225e-02],
        [-1.32636984e-02]]),
 array([[ 0.78374462,  1.0638874 ,  0.47257909,  0.59515403,  0.39969358,
          0.37200897, -0.42478028,  0.26457151],
        [ 0.91302903,  0.74845194,  0.87118199,  0.76607486,  0.76675793,
          0.88750753,  0.95587759,  1.13524311],
        [ 1.29153651,  1.41819833,  1.32442383,  1.43575227,  1.44800873,
          1.4831545 ,  1.48568831,  1.71541832]]),
 array([0, 1, 2, 1, 2, 1, 2, 0, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1,
        2, 2, 1, 1, 2, 0, 0, 2]),
 None,
 1014,
 np.float64(20.326500483282626))

In [9]:
# @njit
def _get_starting_values(y, x, G: int, N: int, K: int):
    """Generates the starting values of theta"""
    num_start_vars: int = x.shape[1] + G  # FIXME I believe that shape is slow in Cython
    random_draws_theta = np.random.choice(N, num_start_vars, replace=False)
    x_stack_start = x[random_draws_theta].reshape(-1, K)
    y_stack_start = y[random_draws_theta].reshape(-1, 1)

    # FIXME some errors may arise, maybe add some checks
    theta_init = lstsq(x_stack_start, y_stack_start, rcond=None)[0]

    random_draws_alpha = np.random.choice(N, size=G, replace=False)
    alpha_init = np.squeeze(y[random_draws_alpha] - x[random_draws_alpha, :, :] @ theta_init)

    return theta_init, alpha_init


@njit
def _compute_groupings(res, alpha):
    """Computes the groupings based on the residuals and alpha"""
    euclidean_distance_between_grouping = ((res[None, :, :] - alpha[:, None, :]) ** 2).sum(axis=2)
    g = np.argmin(euclidean_distance_between_grouping, axis=0)  # Closest group
    return g


# @njit
def _compute_alpha(res, g, G):
    """Computes the alpha values based on the residuals and groupings"""
    counts = np.bincount(g, minlength=G)[:, None]  # (G, 1) — number of elements in each group
    sums = np.zeros((G, res.shape[1]))  # (G, K) — sum of residuals per group
    np.add.at(sums, g, res)  # sums[i] += res[j] for all j where g[j] == i
    alpha = sums / counts  # mean = sum / count
    return alpha


# @njit
def _compute_theta(x, y, alpha, g):
    """Computes the theta values based on the x, y, alpha and groupings"""
    # FIXME check if this makes sense
    K = x.shape[2]  # FIXME I believe that shape is slow in Cython
    theta = lstsq(x.reshape(-1, K), y.reshape(-1, 1) - alpha[g].reshape(-1, 1), rcond=None)[0]
    return theta


# @njit
def _compute_residuals(y, x, theta):
    """Computes the residuals based on y, x and theta"""
    res = np.squeeze(y - x @ theta)
    return res


@njit
def _compute_objective_value(res, alpha, g):
    """Computes the objective value based on the residuals, alpha and groupings"""
    objective_value = ((res - alpha[g]) ** 2).sum()
    return objective_value


def _reorder_groups(g, alpha):
    """Reorders the groups based on the first value of alpha"""
    # FIXME this is not the best way to do this
    # But it works for now
    mapping = np.argsort(alpha[:, 0])
    ordered_g = np.argsort(mapping)[g]
    ordered_alpha = alpha[mapping]
    return ordered_g, ordered_alpha

def _hkmeans(y, x, theta, alpha, g, G, max_iter=1000, tol=1e-6):
    objective_value = np.inf

    for i in range(max_iter):
        res = _compute_residuals(y, x, theta)
        alpha = _compute_alpha(res, g, G)
        theta = _compute_theta(x, y, alpha, g)
        res = _compute_residuals(y, x, theta)
        alpha = _compute_alpha(res, g, G)
        g = _compute_groupings(res, alpha)
        alpha = _compute_alpha(res, g, G)
        new_objective_value = _compute_objective_value(res, alpha, g)

        if abs(objective_value - new_objective_value) < tol:
            return theta, alpha, g, i, new_objective_value

        objective_value = new_objective_value

    return theta, alpha, g, max_iter, objective_value


def _run_vns(y, x, g, G, N, alpha, theta, init_objective_value, max_vns_iter=10, tol=1e-8, max_alg1_iter=20):
    # FIXME this is not the best way to do this
    # But it works for now
    best_objective_value = init_objective_value
    objective_value = np.inf
    g = g.copy()


    i = 1
    while i <= max_vns_iter:
        # Randomly change a few groupings
        g_new = g.copy()

        # FIXME this should check if there are empty groups
        g_new[np.random.choice(N, size=i, replace=False)] = np.random.choice(G, size=i, replace=True)

        # Apply algorithm 1
        theta_new, alpha_new, g_new, iterations_used, objective_value = _hkmeans(y, x, theta, alpha, g_new, G)

        # Local 1 step search
        changed = True
        while changed:
            changed = False
            for j in range(N):
                for k in range(G):
                    if g_new[j] == k:
                        continue

                    # FIXME add check to not leave any group empty

                    g_local = g_new.copy()
                    g_local[j] = k

                    # FIXME this may be very slow
                    if (np.bincount(g_local, minlength=G)==0).any():
                        continue

                    theta_local, alpha_local, g_local, iterations_used, objective_value_local = _hkmeans(y, x, theta_new, alpha_new, g_local, G, max_alg1_iter)

                    if objective_value_local < objective_value:
                        g_new = g_local
                        theta_new = theta_local
                        alpha_new = alpha_local
                        objective_value = objective_value_local
                        changed = True
                        # print(f"Changed group {j} to {k} in iteration {i} with objective value {objective_value}")

        if objective_value < best_objective_value:
            g = g_new
            theta = theta_new
            alpha = alpha_new
            best_objective_value = objective_value
            i = 1

        else:
            i+= 1

    return g, theta, alpha, objective_value


def _grouped_fixed_effects_iteration_vns(y, x, G: int, N: int, K: int, max_iter=10000, tol=1e-8, neighbor_max=10):
    # FIXME possibly create some sort of array that stores these values
    # Could be used for debugging
    theta, alpha = _get_starting_values(y, x, G, N, K)
    res = _compute_residuals(y, x, theta)
    g = _compute_groupings(res, alpha)

    objective_value = np.inf

    for i in range(max_iter):
        alpha = _compute_alpha(res, g, G)
        theta = _compute_theta(x, y, alpha, g)
        res = _compute_residuals(y, x, theta)
        alpha = _compute_alpha(res, g, G)
        g = _compute_groupings(res, alpha)
        alpha = _compute_alpha(res, g, G)
        new_objective_value = _compute_objective_value(res, alpha, g)

        # TODO run VNS
        g, theta, alpha, new_objective_value = _run_vns(y, x, g, G, N, alpha, theta, new_objective_value)

        # if abs(objective_value - new_objective_value) < tol:
        #     iterations_used = i
        #     objective_value = new_objective_value
        #     break

        objective_value = new_objective_value

    return theta, alpha, g, max_iter, objective_value


def _compute_eta(y_bar, x_bar, theta):
    """Computes the eta values based on y_bar, x_bar and theta"""
    eta = np.squeeze(y_bar - x_bar @ theta)
    return eta


# FIXME not used right now but still neccesary
def _compute_statistics(objective_value, N, T, K, G):
    """Computes the statistics based on the objective value, N, T, K and G"""
    # FIXME this is not the best way to do this
    # But it works for now
    sigma_squared = 1 / (N * T - G * T - N - K) * objective_value
    BIC = 1 / (N * T) * objective_value + sigma_squared * (G * T + N + K) / (N * T)
    return sigma_squared, BIC


def grouped_fixed_effects(
    y, x, G, max_iter=10000, tol=1e-8, gfe_iterations=20, unit_specific_effects=False, enable_vns=True
):
    """
    Computes the grouped fixed effects using the algorithm described in the paper.
    """
    # FIXME not really required if unit_specific_effects is False
    # But this seems like the easiest implementation
    x_bar = np.mean(x, axis=1, keepdims=True)
    y_bar = np.mean(y, axis=1, keepdims=True)

    best_theta = None
    best_alpha = None
    best_g = None
    best_iterations_used = None
    best_objective_value = np.inf

    N = x.shape[0]
    K = x.shape[2]

    # FIXME this code does not drop constant binary variables by itself
    # The input class should be able to do this
    if unit_specific_effects:
        # Demean x and y
        x = x - x_bar
        y = y - y_bar

    for _ in range(gfe_iterations):
        if enable_vns:
            theta, alpha, g, iterations_used, objective_value = _grouped_fixed_effects_iteration_vns(
                y,
                x,
                G,
                N,
                K,
                max_iter,
                tol,
            )
        else:
            theta, alpha, g, iterations_used, objective_value = _grouped_fixed_effects_iteration(
                y,
                x,
                G,
                N,
                K,
                max_iter,
                tol,
            )

        if objective_value < best_objective_value:
            best_objective_value = objective_value
            best_theta = theta
            best_g, best_alpha = _reorder_groups(g, alpha)
            # best_g, best_alpha = g, alpha
            best_iterations_used = iterations_used

    if unit_specific_effects:
        eta = _compute_eta(y_bar, x_bar, best_theta)
        return best_theta, best_alpha, best_g, eta, best_iterations_used, best_objective_value

    # NOTE None is for lack of unit specific effects
    return best_theta, best_alpha, best_g, None, best_iterations_used, best_objective_value

# FIXME tolerance is not relevant for VNS
grouped_fixed_effects(y[0:30], x[0:30], 2, max_iter=10, tol=1e-8, gfe_iterations=1, unit_specific_effects=False, enable_vns=True)

(array([[ 1.57168978e-01],
        [-4.40835523e-05],
        [ 1.27102610e-01],
        [ 1.62127299e-02],
        [-6.39706245e-03]]),
 array([[0.97343144, 1.03654322, 0.88829947, 0.79455825, 0.71647648,
         0.77282112, 0.58037268, 0.80762535],
        [1.46456815, 1.43699556, 1.42924016, 1.45917693, 1.41420247,
         1.37946067, 1.3035801 , 1.46244405]]),
 array([0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
        1, 1, 0, 0, 1, 0, 0, 1]),
 None,
 10,
 27.718551774034854)

In [10]:
# A practical rule of thumb for choosing the tuning parameters is to check that different starting values tend to yield the exact same solution. (from the paper)
# grouped_fixed_effects(y[0:60], x[0:60], 3, max_iter=10000, tol=1e-8, gfe_iterations=100)
grouped_fixed_effects(
    y[0:30], x[0:30], 3, max_iter=1000, tol=1e-8, gfe_iterations=100, unit_specific_effects=False, enable_vns=False
)

(array([[ 2.22488343e-01],
        [-1.42335696e-05],
        [-5.81914823e-02],
        [ 4.76984494e-02],
        [-1.25956063e-02]]),
 array([[ 0.73126636,  0.89115714,  0.51139862,  0.61759711,  0.30741283,
          0.24232603, -0.74638001, -0.00795854],
        [ 0.92106121,  0.80920796,  0.79866353,  0.72195016,  0.70963451,
          0.78703147,  0.82873625,  0.97793716],
        [ 1.31643514,  1.42333681,  1.32876313,  1.38804703,  1.38068914,
          1.40903594,  1.36407096,  1.60543939]]),
 array([0, 1, 2, 1, 2, 1, 2, 0, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1,
        2, 2, 1, 1, 2, 1, 0, 1]),
 None,
 905,
 20.43438978682646)

## Attempt at implementing Extension 2

In [10]:
# @njit
# FIXME remove this variable
def _get_starting_values_hetrogeneous(y, x, G: int, N: int, K: int):
    """Generates the starting values of theta"""
    num_start_vars: int = x.shape[1] + G  # FIXME I believe that shape is slow in Cython
    random_draws_theta = np.random.choice(N, num_start_vars, replace=False)
    x_stack_start = x[random_draws_theta].reshape(-1, K)
    y_stack_start = y[random_draws_theta].reshape(-1, 1)

    # FIXME some errors may arise, maybe add some checks
    theta_init = lstsq(x_stack_start, y_stack_start, rcond=None)[0]

    random_draws_alpha = np.random.choice(N, size=G, replace=False)
    alpha_init = np.squeeze(y[random_draws_alpha] - x[random_draws_alpha, :, :] @ theta_init)

    # FIXME this should be changed such that
    return np.tile(theta_init, (1, G)), alpha_init

# TODO create function for hetrogeneous theta


# @njit
def _compute_groupings_hetrogeneous(res, alpha):
    """Computes the groupings based on the residuals and alpha"""
    euclidean_distance_between_grouping = np.squeeze((res - alpha.T[None, :, :]) ** 2).sum(axis=1)
    g = np.argmin(euclidean_distance_between_grouping, axis=1)  # Closest group
    return g


# # @njit
def _compute_alpha_hetrogeneous(res, g, G):
    """Computes the alpha values based on the residuals and groupings"""
    n, T, _ = res.shape
    counts = np.bincount(g, minlength=G)  # shape: (G,)
    R = res[np.arange(n), :, g]  # shape: (n, T)
    sums = np.zeros((G, T))
    np.add.at(sums, g, R)  # sums[k] += R[i] whenever g[i]==k
    alphas = sums / counts[:, None]  # broadcast counts over T
    return alphas


# # @njit
def _compute_theta_hetrogeneous(x, y, alpha, g, G, K):
    """Computes the theta values based on the x, y, alpha and groupings"""
    # FIXME check if this makes sense
    theta = np.zeros((K, G))
    for i in range(G):
        theta[:, i] = lstsq(x[g==i].reshape(-1, K), np.squeeze((np.squeeze(y[g == i]) - alpha[i]).reshape(-1, 1)), rcond=None)[0]

    return theta


# # @njit
def _compute_residuals_hetrogeneous(y, x, theta, G):
    """Computes the residuals based on y, x and theta"""
    res = np.tile(y, (1, G)) - x @ theta
    return res


@njit
def _compute_objective_value_hetrogeneous(res, alpha, g, N):
    """Computes the objective value based on the residuals, alpha and groupings"""
    s = 0
    for i in range(N):
        s += ((res[i,:,g[i]] - alpha[g[i]]) ** 2).sum()

    return s


def _reorder_groups_hetrogeneous(g, alpha, theta):
    """Reorders the groups based on the first value of alpha"""
    # FIXME this is not the best way to do this
    # But it works for now
    mapping = np.argsort(alpha[:, 0])
    ordered_g = np.argsort(mapping)[g]
    ordered_alpha = alpha[mapping]
    ordered_theta = theta[:, mapping]
    return ordered_g, ordered_alpha, ordered_theta


def _hkmeans_hetrogeneous(y, x, theta, alpha, g, G, K, N, max_iter=1000, tol=1e-6):
    objective_value = np.inf

    for i in range(max_iter):
        res = _compute_residuals_hetrogeneous(y, x, theta, G)
        alpha = _compute_alpha_hetrogeneous(res, g, G)
        theta = _compute_theta_hetrogeneous(x, y, alpha, g, G, K)
        res = _compute_residuals_hetrogeneous(y, x, theta, G)
        alpha = _compute_alpha_hetrogeneous(res, g, G)
        g = _compute_groupings_hetrogeneous(res, alpha)
        alpha = _compute_alpha_hetrogeneous(res, g, G)
        new_objective_value = _compute_objective_value_hetrogeneous(res, alpha, g, N)

        if abs(objective_value - new_objective_value) < tol:
            return theta, alpha, g, i, new_objective_value

        objective_value = new_objective_value

    return theta, alpha, g, max_iter, objective_value


def _run_vns_hetrogeneous(y, x, g, G, N, alpha, theta, init_objective_value, K, max_vns_iter=10, tol=1e-8, max_alg1_iter=20):
    # FIXME this is not the best way to do this
    # But it works for now
    best_objective_value = init_objective_value
    objective_value = np.inf
    g = g.copy()

    i = 1
    while i <= max_vns_iter:
        print(f"vns iter: {i}")
        # Randomly change a few groupings
        g_new = g.copy()

        # FIXME this should check if there are empty groups
        g_new[np.random.choice(N, size=i, replace=False)] = np.random.choice(G, size=i, replace=True)

        # Apply algorithm 1
        theta_new, alpha_new, g_new, iterations_used, objective_value = _hkmeans_hetrogeneous(
            y, x, theta, alpha, g_new, G, K, N
        )

        # Local 1 step search
        changed = True
        while changed:
            changed = False
            for j in range(N):
                for k in range(G):
                    if g_new[j] == k:
                        continue

                    # FIXME add check to not leave any group empty

                    g_local = g_new.copy()
                    g_local[j] = k

                    # FIXME this may be very slow
                    if (np.bincount(g_local, minlength=G) == 0).any():
                        continue

                    theta_local, alpha_local, g_local, iterations_used, objective_value_local = _hkmeans_hetrogeneous(
                        y, x, theta_new, alpha_new, g_local, G, K, N, max_alg1_iter
                    )

                    if objective_value_local < objective_value:
                        g_new = g_local
                        theta_new = theta_local
                        alpha_new = alpha_local
                        objective_value = objective_value_local
                        changed = True
                        # print(f"Changed group {j} to {k} in iteration {i} with objective value {objective_value}")

        if objective_value < best_objective_value:
            g = g_new
            theta = theta_new
            alpha = alpha_new
            best_objective_value = objective_value
            i = 1
            print(f"Succes at iteration {i} with objective value {best_objective_value}")

        else:
            i += 1

    return g, theta, alpha, objective_value


def _grouped_fixed_effects_iteration_vns_hetrogeneous(y, x, G: int, N: int, K: int, max_iter=10000, tol=1e-8, neighbor_max=10):
    # FIXME possibly create some sort of array that stores these values
    # Could be used for debugging
    theta, alpha = _get_starting_values_hetrogeneous(y, x, G, N, K)
    res = _compute_residuals_hetrogeneous(y, x, theta, G)
    g = _compute_groupings_hetrogeneous(res, alpha)

    objective_value = np.inf

    for i in range(max_iter):
        print(f"ITERATION: {i}")
        alpha = _compute_alpha_hetrogeneous(res, g, G)
        theta = _compute_theta_hetrogeneous(x, y, alpha, g, G, K)
        res = _compute_residuals_hetrogeneous(y, x, theta, G)
        alpha = _compute_alpha_hetrogeneous(res, g, G)
        g = _compute_groupings_hetrogeneous(res, alpha)
        alpha = _compute_alpha_hetrogeneous(res, g, G)
        new_objective_value = _compute_objective_value_hetrogeneous(res, alpha, g, N)

        # TODO run VNS
        g, theta, alpha, new_objective_value = _run_vns_hetrogeneous(y, x, g, G, N, alpha, theta, new_objective_value, K)

        # if abs(objective_value - new_objective_value) < tol:
        #     iterations_used = i
        #     objective_value = new_objective_value
        #     break

        objective_value = new_objective_value

    return theta, alpha, g, max_iter, objective_value


def _grouped_fixed_effects_iteration_hetrogeneous(y, x, G: int, N: int, K: int, max_iter=10000, tol=1e-8):
    # FIXME possibly create some sort of array that stores these values
    # Could be used for debugging
    theta, alpha = _get_starting_values_hetrogeneous(y, x, G, N, K)
    res = _compute_residuals_hetrogeneous(y, x, theta, G)
    g = _compute_groupings_hetrogeneous(res, alpha)

    objective_value = np.inf

    iterations_used = 0
    for i in range(max_iter):
        alpha = _compute_alpha_hetrogeneous(res, g, G)
        theta = _compute_theta_hetrogeneous(x, y, alpha, g, G, K)
        res = _compute_residuals_hetrogeneous(y, x, theta, G)
        alpha = _compute_alpha_hetrogeneous(res, g, G)
        g = _compute_groupings_hetrogeneous(res, alpha)
        new_objective_value = _compute_objective_value_hetrogeneous(res, alpha, g, N)

        if abs(objective_value - new_objective_value) < tol:
            iterations_used = i
            objective_value = new_objective_value
            break

        objective_value = new_objective_value

    return theta, alpha, g, iterations_used, objective_value


def _compute_eta_hetrogeneous(y_bar, x_bar, theta):
    """Computes the eta values based on y_bar, x_bar and theta"""
    eta = np.squeeze(y_bar - x_bar @ theta.mean(axis=1))
    return eta


# # FIXME not used right now but still neccesary
# def _compute_statistics(objective_value, N, T, K, G):
#     """Computes the statistics based on the objective value, N, T, K and G"""
#     # FIXME this is not the best way to do this
#     # But it works for now
#     sigma_squared = 1 / (N * T - G * T - N - K) * objective_value
#     BIC = 1 / (N * T) * objective_value + sigma_squared * (G * T + N + K) / (N * T)
#     return sigma_squared, BIC


def grouped_fixed_effects(
    y, x, G, max_iter=10000, tol=1e-8, gfe_iterations=20, unit_specific_effects=False, enable_vns=True, hetrogeneous_theta=False
):
    """
    Computes the grouped fixed effects using the algorithm described in the paper.
    """
    # FIXME not really required if unit_specific_effects is False
    # But this seems like the easiest implementation
    x_bar = np.mean(x, axis=1, keepdims=True)
    y_bar = np.mean(y, axis=1, keepdims=True)

    best_theta = None
    best_alpha = None
    best_g = None
    best_iterations_used = None
    best_objective_value = np.inf

    N = x.shape[0]
    T = x.shape[1]
    K = x.shape[2]

    # FIXME this code does not drop constant binary variables by itself
    # The input class should be able to do this
    if unit_specific_effects:
        # Demean x and y
        x = x - x_bar
        y = y - y_bar

    for _ in range(gfe_iterations):
        # FIXME should be better way to do this
        if hetrogeneous_theta and enable_vns:
            theta, alpha, g, iterations_used, objective_value = _grouped_fixed_effects_iteration_vns_hetrogeneous(
                y,
                x,
                G,
                N,
                K,
                max_iter,
                tol,
            )
        elif enable_vns:
            theta, alpha, g, iterations_used, objective_value = _grouped_fixed_effects_iteration_vns(
                y,
                x,
                G,
                N,
                K,
                max_iter,
                tol,
            )

        elif hetrogeneous_theta:
            theta, alpha, g, iterations_used, objective_value = _grouped_fixed_effects_iteration_hetrogeneous(
                y,
                x,
                G,
                N,
                K,
                max_iter,
                tol,
            )

        else:
            theta, alpha, g, iterations_used, objective_value = _grouped_fixed_effects_iteration(
                y,
                x,
                G,
                N,
                K,
                max_iter,
                tol
            )

        if objective_value < best_objective_value:
            best_objective_value = objective_value
            best_g, best_alpha, best_theta = _reorder_groups_hetrogeneous(g, alpha, theta)
            # best_g, best_alpha = g, alpha
            best_iterations_used = iterations_used

    # FIXME this should not be true for hetrogeneous theta
    # Because does not work
    if unit_specific_effects:
        eta = _compute_eta_hetrogeneous(y_bar, x_bar, best_theta)
        return best_theta, best_alpha, best_g, eta, best_iterations_used, best_objective_value

    # NOTE None is for lack of unit specific effects
    return best_theta, best_alpha, best_g, None, best_iterations_used, best_objective_value


# # FIXME tolerance is not relevant for VNS
# FIXME for hetrogeneous theta VNS is super super slow
grouped_fixed_effects(
    y[0:30], x[0:30], 3, max_iter=1000, tol=0, gfe_iterations=100, unit_specific_effects=False, enable_vns=False, hetrogeneous_theta=True
)

(array([[ 4.86008134e-01,  5.18001615e-02, -1.99441828e-01],
        [-8.13075863e-05, -6.44077350e-05, -4.02078570e-04],
        [ 6.01876007e-02,  1.51317580e-01, -2.49800181e-16],
        [-7.58122222e-03, -3.00986592e-02,  4.56324210e-01],
        [-2.62659555e-02, -3.83471825e-03, -3.16578091e-02]]),
 array([[ 0.33446786,  0.15809625, -0.10072759, -0.35208115, -0.47542558,
         -0.4046256 , -0.37846026, -0.27071044],
        [ 1.78426222,  1.84229201,  1.9234376 ,  2.0195686 ,  2.04513436,
          2.07679585,  2.06409952,  2.28386173],
        [ 2.39490207,  2.85697147,  3.34480934,  4.02789088,  4.48089238,
          4.77309454,  4.08219583,  6.07411512]]),
 array([2, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
        1, 1, 0, 0, 1, 0, 2, 1]),
 None,
 0,
 17.192112255108622)